# Running the backend

## Running the PostgreSQL Docker Container

In [92]:
%%bash
docker compose up -d

 Network cookbook-app_default Creating 
 Network cookbook-app_default Created 
 Container cookbook_db Creating 
 Container cookbook_db Created 
 Container cookbook_db Starting 
 Container cookbook_db Started 


In [94]:
%%bash
docker ps

CONTAINER ID   IMAGE         COMMAND                  CREATED          STATUS          PORTS                                         NAMES
a0692cc2bb5c   postgres:15   "docker-entrypoint.s…"   14 seconds ago   Up 14 seconds   0.0.0.0:5433->5432/tcp, [::]:5433->5432/tcp   cookbook_db


## Stopping the container

In [ ]:
%%bash
docker compose down

 Container cookbook_db Stopping 
 Container cookbook_db Stopped 
 Container cookbook_db Removing 
 Container cookbook_db Removed 
 Network cookbook-app_default Removing 
 Network cookbook-app_default Removed 


## Inspecting the PostgreSQL container

```bash
docker exec -it <CONTAINER_ID> psql -U user -d cookbook 
```


### All DB Tables
```sql
SELECT table_catalog, table_schema, table_name, table_type, is_insertable_into , is_typed  FROM information_schema.tables t  WHERE t.table_schema LIKE 'public' ORDER BY table_name;
 table_catalog | table_schema |      table_name       | table_type | is_insertable_into | is_typed 
---------------+--------------+-----------------------+------------+--------------------+----------
 cookbook      | public       | diets                 | BASE TABLE | YES                | NO
 cookbook      | public       | preferences           | BASE TABLE | YES                | NO
 cookbook      | public       | recipes               | BASE TABLE | YES                | NO
 cookbook      | public       | menu_plan             | BASE TABLE | YES                | NO
 cookbook      | public       | nutrients             | BASE TABLE | YES                | NO
 cookbook      | public       | ingredients           | BASE TABLE | YES                | NO
 cookbook      | public       | mise_en_place         | BASE TABLE | YES                | NO
 cookbook      | public       | recipe_nutrition_view | VIEW       | NO                 | NO
```

## Initialize the DB with the Seed Script

In [95]:
%%bash
cd backend
source venv/bin/activate
python -m app.seed

Successfully seeded all tables!


## Running FastAPI

In [42]:
%%bash
cd backend
source venv/bin/activate
nohup $(uvicorn app.main:app --reload > log.txt 2>&1) &

nohup: missing operand
Try 'nohup --help' for more information.


In [ ]:
%%bash
kill -9 558397
ps -ef | grep uvicorn

iissa     558397       1 43 10:58 ?        00:00:52 /home/iissa/Workspace/Dev/cookbook-app/backend/venv/bin/python3 /home/iissa/Workspace/Dev/cookbook-app/backend/venv/bin/uvicorn app.main:app --reload
iissa     559231  559228  0 11:00 ?        00:00:00 grep uvicorn


# Views

## V_Ingredients_Nutrients


```sql
SELECT  
    name , 
    unit_default, 
    i.group, 
    kcal, 
    kj, 
    fat, 
    saturated_fatty_acids, 
    mono_unsaturated_fatty_acids, 
    polyunsaturated_fatty_acids, 
    cholesterol_mg, 
    carbohydrates, 
    sugar, 
    starch, 
    dietary_fibre, 
    protein, 
    salt 
FROM ingredients i 
JOIN nutrients n ON i.group = n.ingredient_group;

              name              | unit_default |      group       | ingredient_group | kcal  |  kj   | fat  | saturated_fatty_acids | mono_unsaturated_fatty_acids | polyunsaturated_fatty_acids | cholesterol_mg | carbohydrates | sugar | starch | dietary_fibre | protein | salt 
--------------------------------+--------------+------------------+------------------+-------+-------+------+-----------------------+------------------------------+-----------------------------+----------------+---------------+-------+--------+---------------+---------+------
 anchovy                        | fillets      | anchovy          | anchovy          |   182 |   765 |  8.5 |                   2.9 |                          3.4 |                         1.3 |             76 |           0.2 |   0.1 |      0 |             0 |    26.4 |   12
 apples                         | pieces       | apple            | apple            |    52 |   222 |  0.3 |                   0.1 |                            0 |                         0.2 |              0 |          11.6 |  11.5 |    0.1 |           1.4 |     0.3 |    0
 arborio rice                   | cups         | rice             | rice             |   352 |  1490 |  0.9 |                   0.2 |                          0.3 |                         0.3 |              0 |            78 |   0.2 |   77.8 |             1 |     7.4 |    0
 artichoke hearts               | g            | artichoke        | artichoke        |    27 |   114 |  0.5 |                     0 |                            0 |                           0 |              0 |           2.3 |   1.2 |    0.4 |           3.2 |     1.8 |  0.8
 aubergine                      | pieces       | aubergine        | aubergine        |    23 |    96 |  0.2 |                     0 |                            0 |                         0.1 |              0 |           3.1 |   2.8 |    0.3 |           2.4 |       1 |    0
(198 rows)
```

## V_Instructions

recipe_name, ingredient_name, group , unit, quantity , preparation
```sql
SELECT 
    recipe_name     AS recipe, 
    ingredient_name AS ingredient, 
    i.group         AS group,  
    unit            AS unit, 
    quantity        AS quantity, 
    preparation     AS preparation
FROM mise_en_place m 
JOIN ingredients i ON 
    i.name = m.ingredient_name 
    AND i.unit_default = m.unit 
;


                 recipe                  |           ingredient           |      group       |  unit   | quantity |                                     preparation                     
-----------------------------------------+--------------------------------+------------------+---------+----------+--------------------------------------------------------------------------------------
 Artichoke Spinach Lasagna               | lasagna noodles                | pasta            | pieces  |        9 | uncooked
 Artichoke Spinach Lasagna               | onion                          | onion            | pieces  |        1 | chopped
 Artichoke Spinach Lasagna               | garlic                         | garlic           | cloves  |        4 | chopped
 Artichoke Spinach Lasagna               | vegetable broth                | vegetable broth  | ml      |      400 | 
 Artichoke Spinach Lasagna               | rosemary                       | herb             | tbsp    |        1 | chopped, fresh
 Artichoke Spinach Lasagna               | artichoke hearts               | artichoke        | g       |      400 | marinated, drained and chopped
 Artichoke Spinach Lasagna               | spinach                        | spinach          | g       |      300 | chopped, thawed, drained and squeezed" dry
 Artichoke Spinach Lasagna               | tomato sauce                   | tomato sauce     | g       |      800 | 
 Artichoke Spinach Lasagna               | mozzarella                     | mozzarella       | g       |      600 | shredded, divided
 Artichoke Spinach Lasagna               | feta                           | feta             | g       |      100 | with herb and garlic , crumbled
 Caldeirada(Portuguese Fish Stew)        | olive oil                      | olive oil        | tbsp    |        3 | 
 Caldeirada(Portuguese Fish Stew)        | onion                          | onion            | pieces  |        1 | diced
 Caldeirada(Portuguese Fish Stew)        | garlic                         | garlic           | cloves  |        4 | rough chopped
 Caldeirada(Portuguese Fish Stew)        | tomatoes                       | tomato           | pieces  |        2 | diced
 Caldeirada(Portuguese Fish Stew)        | white wine                     | wine             | cups    |     0.25 | 
 Caldeirada(Portuguese Fish Stew)        | chicken stock                  | chicken broth    | cups    |        4 | (or fish stock, or use half clam juice)
 Caldeirada(Portuguese Fish Stew)        | fish sauce                     | fish sauce       | tsp     |        2 | (skip if using fish stock/clam juice)
 Caldeirada(Portuguese Fish Stew)        | saffron                        | spice            | pinches |        1 | crumbled
 Caldeirada(Portuguese Fish Stew)        | ground clove (clou de girofle) | spice            | tsp     |    0.125 | Ground, (or allspice) - optional
 Caldeirada(Portuguese Fish Stew)        | paprika                        | paprika          | tsp     |     0.25 | Smoked
 Caldeirada(Portuguese Fish Stew)        | salt                           | salt             | tsp     |     0.25 | 
 Caldeirada(Portuguese Fish Stew)        | pepper                         | pepper           | tsp     |     0.25 | 
 Caldeirada(Portuguese Fish Stew)        | bay                            | bay              | leaves  |        2 | 
 Caldeirada(Portuguese Fish Stew)        | thyme                          | herb             | tbsp    |        2 | (Fresh lemon thyme is nice)
 Caldeirada(Portuguese Fish Stew)        | potatoes                       | potato           | g       |      500 | cut into 1-inch cubes
 Caldeirada(Portuguese Fish Stew)        | fresh fish fillet              | fish             | g       |     1000 | (boneless, skinless) wild cod, haddock, sea bass, salmon, cut into 1 1/2-inch pieces
 Cheesy Beef and Bean Burritos           | ground beef                    | beef             | g       |     1500 | 
 Cheesy Beef and Bean Burritos           | taco seasoning                 | seasoning        | cups    |     0.33 | 
 Cheesy Beef and Bean Burritos           | green chile                    | chili            | g       |      100 | Chopped, or salsa
 Cheesy Beef and Bean Burritos           | refried beans                  | kidney beans     | g       |      800 | 
 Cheesy Beef and Bean Burritos           | cilantro leaves                | herb             | cups    |     0.33 | Chopped, fresh
 Cheesy Beef and Bean Burritos           | mexican cheese blend           | cheese           | cups    |        3 | Shredded
 Cheesy Beef and Bean Burritos           | tortillas                      | bread            | pieces  |        8 | Large flour tortillas
 Cheesy Beef and Bean Burritos           | sour cream                     | sour cream       | package |        1 | 
 Cheesy Beef and Bean Burritos           | avocado                        | avocado          | pieces  |        1 | slices
 Cheesy Beef and Bean Burritos           | lettuce                        | lettuce          | pieces  |        1 | Shredded
 Cheesy Beef and Bean Burritos           | tomatoes                       | tomato           | pieces  |        2 | Chopped
 Cheesy Beef and Bean Burritos           | jalapeno                       | chili            | pieces  |        4 | slices
 Cheesy Beef and Bean Burritos           | black olives                   | olives           | cups    |        1 | Sliced
 Chicken Risotto                         | onion                          | onion            | cups    |        1 | cut into large chunks
 Chicken Risotto                         | carrot                         | carrot           | cups    |        1 | peeled, cut into large chunks
 Chicken Risotto                         | celery                         | celery           | cups    |        1 | cut into large chunks
 Chicken Risotto                         | garlic                         | garlic           | cloves  |        2 | large, peeled
 Chicken Risotto                         | kosher salt                    | salt             | tsp     |        1 | divided
 Chicken Risotto                         | olive oil                      | olive oil        | cups    |     0.25 | 
 Chicken Risotto                         | chicken thighs                 | chicken          | g       |     1000 | skinless, Boneless (cut into bite-sized pieces)
 Chicken Risotto                         | bay                            | bay              | leaves  |        2 | dried
 Chicken Risotto                         | dry white wine                 | wine             | cups    |        1 | 
 Chicken Risotto                         | chicken broth                  | chicken broth    | cups    |        5 | Hot, low sodium
 Chicken Risotto                         | arborio rice                   | rice             | cups    |        2 | 
 Chicken Risotto                         | butter                         | butter           | tbsp    |        3 | 
 Chicken Risotto                         | parmesan                       | cheese           | cups    |      0.5 | shredded
 Chicken Risotto                         | parsley                        | parsley          | cups    |     0.25 | chopped
 Classic Minestrone Soup                 | olive oil                      | olive oil        | tbsp    |        1 | 
 Classic Minestrone Soup                 | onion                          | onion            | cups    |        1 | Diced
 Classic Minestrone Soup                 | carrot                         | carrot           | cups    |     0.75 | Diced
 Classic Minestrone Soup                 | celery                         | celery           | cups    |     0.75 | Sliced
 Classic Minestrone Soup                 | garlic                         | garlic           | tsp     |        1 | Minced
 Classic Minestrone Soup                 | rosemary                       | herb             | tsp     |      1.5 | Minced fresh
 Classic Minestrone Soup                 | tomato sauce                   | tomato sauce     | g       |      425 | 
 Classic Minestrone Soup                 | cannellini beans               | kidney beans     | g       |      425 | rinsed and drained
 Classic Minestrone Soup                 | water                          | water            | cups    |      2.5 | 
 Classic Minestrone Soup                 | chicken broth                  | chicken broth    | cups    |        2 | 
 Classic Minestrone Soup                 | zucchini                       | zucchini         | cups    |        1 | Diced
 Classic Minestrone Soup                 | italian pasta                  | pasta            | cups    |      0.5 | 
 Classic Minestrone Soup                 | black pepper                   | pepper           | tsp     |     0.25 | 
 Classic Minestrone Soup                 | parsley                        | parsley          | cups    |     0.25 | Chopped fresh
 Classic Minestrone Soup                 | parmesan                       | cheese           | tbsp    |        1 | Shredded
 Cod with Garlicky Tomatoes and Potatoes | olive oil                      | olive oil        | tbsp    |        2 | Extra-virgin
 Cod with Garlicky Tomatoes and Potatoes | garlic                         | garlic           | cloves  |        3 | smashed and peeled
 Cod with Garlicky Tomatoes and Potatoes | baby potatoes                  | potato           | g       |      500 | sliced into 1cm-thick rounds
 Cod with Garlicky Tomatoes and Potatoes | dry white wine                 | wine             | cups    |      0.5 | such as pinot grigio
 Cod with Garlicky Tomatoes and Potatoes | diced tomatoes                 | tomato           | g       |      400 | 
 Cod with Garlicky Tomatoes and Potatoes | paprika                        | paprika          | tsp     |      0.5 | Smoked or regular
 Cod with Garlicky Tomatoes and Potatoes | kosher salt                    | salt             | tsp     |     0.75 | divided
 Cod with Garlicky Tomatoes and Potatoes | black pepper                   | pepper           | tsp     |     0.25 | divided
 Cod with Garlicky Tomatoes and Potatoes | red pepper                     | paprika          | tsp     |     0.25 | Crushed flakes
 Cod with Garlicky Tomatoes and Potatoes | cod                            | cod              | g       |      700 | Skinless fillet
 Cod with Garlicky Tomatoes and Potatoes | lemon                          | lemon            | pieces  |        1 | quartered
 Cod with Garlicky Tomatoes and Potatoes | parsley                        | parsley          | cups    |     0.25 | Chopped fresh
 Crêpes sucrées                          | farine                         | flour            | g       |      250 | 
 Crêpes sucrées                          | sucre                          | sugar            | g       |       50 | 
 Crêpes sucrées                          | sel                            | salt             | pinches |        1 | 
 Crêpes sucrées                          | oeufs                          | egg              | pieces  |        4 | 
 Crêpes sucrées                          | lait                           | milk             | ml      |      500 | 
 Crêpes sucrées                          | beurre                         | butter           | g       |       50 | fondu
 Crêpes sucrées                          | extrait de vanille             | vanille          | tsp     |        1 | facultatif
 Crêpes sucrées                          | citron                         | lemon            | zest    |        1 | facultatif
 Galette bretonne                        | farine de sarrasin             | flour            | g       |      330 | 
 Galette bretonne                        | eau                            | water            | ml      |      750 | 
 Galette bretonne                        | sel                            | salt             | tsp     |        1 | 
 Galette bretonne                        | jambon                         | ham              | slice   |        1 | 
 Galette bretonne                        | oeuf                           | egg              | pieces  |        1 | 
 Galette bretonne                        | fromage râpé                   | cheese           | g       |       50 | (emmental, comté, etc.)
 Galette bretonne                        | beurre                         | butter           | tbsp    |        1 | pour la cuisson
 Hackfleisch Zucchini Omelette           | garlic                         | garlic           | cloves  |        1 | 
 Hackfleisch Zucchini Omelette           | oil                            | oil              | tbsp    |        1 | 
 Hackfleisch Zucchini Omelette           | ground beef                    | beef             | g       |      500 | beef mince
 Hackfleisch Zucchini Omelette           | salt                           | salt             | tsp     |      0.5 | 
 Hackfleisch Zucchini Omelette           | pepper                         | pepper           | pinches |        1 | 
 Hackfleisch Zucchini Omelette           | zucchini                       | zucchini         | g       |      300 | 
 Hackfleisch Zucchini Omelette           | eggs                           | egg              | pieces  |        8 | Fresh
 Hackfleisch Zucchini Omelette           | grated cheese                  | cheese           | g       |       50 | 
 Hackfleisch Zucchini Omelette           | cayenne pepper                 | pepper           | tsp     |      0.5 | 
 Homemade Lasagna                        | ground beef                    | beef             | g       |      500 | Lean
 Homemade Lasagna                        | onion                          | onion            | cups    |      0.5 | Minced
 Homemade Lasagna                        | tomatoes                       | tomato           | g       |      800 | Crushed
 Homemade Lasagna                        | tomato sauce                   | tomato sauce     | g       |      225 | 
 Homemade Lasagna                        | parsley                        | parsley          | tbsp    |        2 | Fresh, chopped, divided
 Homemade Lasagna                        | garlic                         | garlic           | cloves  |        1 | crushed
 Homemade Lasagna                        | dried basil                    | basil            | tsp     |      1.5 | 
 Homemade Lasagna                        | dried oregano                  | oregano          | tsp     |      0.5 | 
 Homemade Lasagna                        | white sugar                    | sugar            | tsp     |    0.125 | 
 Homemade Lasagna                        | lasagna noodles                | pasta            | g       |      450 | 
 Homemade Lasagna                        | cottage cheese                 | cottage cheese   | g       |      500 | 
 Homemade Lasagna                        | parmesan                       | cheese           | cups    |     0.75 | Grated
 Homemade Lasagna                        | eggs                           | egg              | pieces  |        3 | 
 Homemade Lasagna                        | salt                           | salt             | tsp     |        2 | 
 Homemade Lasagna                        | ground black pepper            | pepper           | tsp     |     0.25 | 
 Homemade Lasagna                        | mozzarella                     | mozzarella       | g       |      450 | Shredded
 Moroccan Lentil Stew                    | water                          | water            | cups    |        5 | Hot
 Moroccan Lentil Stew                    | tomato bouillon                | broth            | package |      0.2 | with chicken flavoring (such as Knorr®)
 Moroccan Lentil Stew                    | olive oil                      | olive oil        | tbsp    |        1 | Extra-virgin
Moroccan Lentil Stew                    | yellow onion                   | onion            | cups    |        1 | chopped
 Moroccan Lentil Stew                    | garlic                         | garlic           | cloves  |        2 | minced
 Moroccan Lentil Stew                    | fenugreek leaves               | herb             | tsp     |        1 | Dried
 Moroccan Lentil Stew                    | bay                            | bay              | leaves  |        1 | 
 Moroccan Lentil Stew                    | ground cumin                   | spice            | tsp     |      0.5 | 
 Moroccan Lentil Stew                    | ground coriander               | spice            | tsp     |      0.5 | 
 Moroccan Lentil Stew                    | ground cinnamon                | cinnamon         | tsp     |      0.5 | 
 Moroccan Lentil Stew                    | salt                           | salt             | tsp     |      0.5 | 
 Moroccan Lentil Stew                    | cayenne pepper                 | pepper           | tsp     |     0.25 | 
 Moroccan Lentil Stew                    | lentils                        | dried lentil     | cups    |        1 | 
 Moroccan Lentil Stew                    | butternut squash               | butternut squash | cups    |        1 | peeled and chopped
 Moroccan Lentil Stew                    | carrot                         | carrot           | pieces  |        1 | large, chopped
 Moroccan Lentil Stew                    | celery stalk                   | celery           | pieces  |        1 | chopped
 Moroccan Lentil Stew                    | green beans                    | green beans      | cups    |      0.5 | chopped
 Moroccan Lentil Stew                    | frozen peas                    | peas             | cups    |     0.25 | 
 Moroccan Lentil Stew                    | cilantro leaves                | herb             | tbsp    |        1 | Fresh, chopped (Optional)
 No Noodle Zucchini Lasagna              | zucchini                       | zucchini         | pieces  |        2 | large
 No Noodle Zucchini Lasagna              | salt                           | salt             | tbsp    |        1 | 
 No Noodle Zucchini Lasagna              | ground beef                    | beef             | g       |      500 | 
 No Noodle Zucchini Lasagna              | ground black pepper            | pepper           | tsp     |      1.5 | 
 No Noodle Zucchini Lasagna              | green bell pepper              | bell pepper      | pieces  |        1 | small, diced
 No Noodle Zucchini Lasagna              | onion                          | onion            | pieces  |        1 | diced
 No Noodle Zucchini Lasagna              | tomato sauce                   | tomato sauce     | g       |      450 | 
 No Noodle Zucchini Lasagna              | tomato paste                   | tomato paste     | cups    |        1 | 
 No Noodle Zucchini Lasagna              | red wine                       | wine             | cups    |     0.25 | 
 No Noodle Zucchini Lasagna              | basil                          | basil            | tbsp    |        2 | Fresh, chopped
 No Noodle Zucchini Lasagna              | oregano                        | oregano          | tbsp    |        1 | Fresh, chopped
 No Noodle Zucchini Lasagna              | water                          | water            | tbsp    |        3 | Hot, as needed
 No Noodle Zucchini Lasagna              | ricotta                        | ricotta          | g       |      450 | Low-fat
 No Noodle Zucchini Lasagna              | egg                            | egg              | pieces  |        1 | 
 No Noodle Zucchini Lasagna              | parsley                        | parsley          | tbsp    |        2 | Fresh, chopped
 No Noodle Zucchini Lasagna              | spinach                        | spinach          | g       |      450 | chopped, thawed and drained
 No Noodle Zucchini Lasagna              | mushrooms                      | mushroom         | g       |      500 | Fresh, sliced
 No Noodle Zucchini Lasagna              | mozzarella                     | mozzarella       | g       |      200 | Shredded
 No Noodle Zucchini Lasagna              | parmesan                       | cheese           | g       |      200 | Grated
 Omelet                                  | eggs                           | egg              | pieces  |        2 | large
 Omelet                                  | salt                           | salt             | pinches |        1 | 
 Omelet                                  | butter                         | butter           | tbsp    |        1 | Unsalted
 Omelet                                  | cheese                         | cheese           | tbsp    |        2 | Grated, any kind
 Omelet                                  | cherry tomatoes                | tomato           | pieces  |        4 | cut in half and sprinkled lightly with salt
 Pizza Margherita                        | flour                          | flour            | g       |      300 | Pizza flour
 Pizza Margherita                        | salt                           | salt             | tsp     |        1 | 
 Pizza Margherita                        | sugar                          | sugar            | tsp     |        1 | 
 Pizza Margherita                        | dry yeast                      | yeast            | g       |        7 | 
 Pizza Margherita                        | water                          | water            | ml      |      180 | Warm
 Pizza Margherita                        | olive oil                      | olive oil        | tbsp    |        1 | 
 Pizza Margherita                        | tomato sauce                   | tomato sauce     | g       |      200 | 
 Pizza Margherita                        | dried oregano                  | oregano          | tsp     |        1 | 
 Pizza Margherita                        | mozzarella                     | mozzarella       | g       |      200 | sliced
 Pizza Margherita                        | basil                          | basil            | leaves  |       15 | Fresh
 Pizza Margherita                        | salt                           | salt             | pinches |        1 | 
 Pizza Margherita                        | pepper                         | pepper           | pinches |        1 | 
 Pizza Quatre Fromages                   | pizza flour                    | flour            | g       |      300 | 
 Pizza Quatre Fromages                   | salt                           | salt             | tsp     |        1 | 
 Pizza Quatre Fromages                   | sugar                          | sugar            | tsp     |        1 | 
 Pizza Quatre Fromages                   | dry yeast                      | yeast            | g       |        7 | 
 Pizza Quatre Fromages                   | water                          | water            | ml      |      180 | Warm
 Pizza Quatre Fromages                   | tomato sauce                   | tomato sauce     | g       |      200 | 
 Pizza Quatre Fromages                   | dried oregano                  | oregano          | tsp     |        1 | 
 Pizza Quatre Fromages                   | olive oil                      | olive oil        | tbsp    |        2 | 
 Pizza Quatre Fromages                   | mozzarella                     | mozzarella       | g       |      100 | sliced
Pizza Quatre Fromages                   | gorgonzola                     | gorgonzola       | g       |      100 | crumbled
 Pizza Quatre Fromages                   | goat cheese                    | goat cheese      | g       |      100 | sliced
 Pizza Quatre Fromages                   | parmesan                       | cheese           | g       |      100 | grated
 Pizza Quatre Fromages                   | pepper                         | pepper           | pinches |        1 | 
 Pizza Quatre Fromages                   | salt                           | salt             | pinches |        1 | 
 Poached Cod in Tomato Curry             | virgin coconut oil             | oil              | tbsp    |        3 | or vegetable oil
 Poached Cod in Tomato Curry             | red chile                      | chili            | pieces  |        1 | halved, seeded, thinly sliced
 Poached Cod in Tomato Curry             | ginger                         | ginger           | cubes   |        2 | 2cm piece, Fresh, peeled, thinly sliced
 Poached Cod in Tomato Curry             | garlic                         | garlic           | cloves  |        2 | thinly sliced
 Poached Cod in Tomato Curry             | cardamom                       | spice            | pods    |        4 | crushed
 Poached Cod in Tomato Curry             | ground coriander               | spice            | tsp     |        1 | 
 Poached Cod in Tomato Curry             | ground turmeric                | spice            | tsp     |      0.5 | 
 Poached Cod in Tomato Curry             | cherry tomatoes                | tomato           | g       |      600 | 
 Poached Cod in Tomato Curry             | coconut cream                  | coconut cream    | cups    |     0.25 | Unsweetened
 Poached Cod in Tomato Curry             | salt                           | salt             | pinches |        1 | Kosher
 Poached Cod in Tomato Curry             | cod                            | cod              | g       |      150 | Skinless fillets
 Poached Cod in Tomato Curry             | basil leaves                   | basil            | cups    |        1 | torn if large
 Portuguese Cod Stew                     | cod                            | cod              | g       |     1000 | Desalted
 Portuguese Cod Stew                     | tomatoes                       | tomato           | pieces  |        4 | medium size, ripe
 Portuguese Cod Stew                     | onion                          | onion            | pieces  |        1 | large size
 Portuguese Cod Stew                     | olive oil                      | olive oil        | ml      |      100 | 
 Portuguese Cod Stew                     | white wine                     | wine             | ml      |      100 | 
 Portuguese Cod Stew                     | pepper                         | pepper           | pinches |        1 | to taste
 Portuguese Cod Stew                     | potatoes                       | potato           | kg      |        1 | Small
 Portuguese Cod Stew                     | bell pepper                    | bell pepper      | pieces  |        1 | Red or Green
 Portuguese Cod Stew                     | garlic                         | garlic           | cloves  |        3 | 
 Portuguese Cod Stew                     | salt                           | salt             | pinches |        1 | to taste
 Poulet Curry                            | chicken                        | chicken          | fillets |        4 | cut into pieces
 Poulet Curry                            | onion                          | onion            | pieces  |        1 | thinly sliced
 Poulet Curry                            | garlic                         | garlic           | cloves  |        2 | chopped
 Poulet Curry                            | ginger                         | ginger           | tbsp    |        1 | Fresh, grated
 Poulet Curry                            | olive oil                      | olive oil        | tbsp    |        2 | 
 Poulet Curry                            | curry powder                   | seasoning        | tbsp    |        2 | 
 Poulet Curry                            | turmeric powder                | spice            | tsp     |        1 | 
 Poulet Curry                            | ground cumin                   | spice            | tsp     |        1 | 
 Poulet Curry                            | ground coriander               | spice            | tsp     |        1 | 
 Poulet Curry                            | coconut milk                   | coconut milk     | ml      |      400 | 
 Poulet Curry                            | salt                           | salt             | pinches |        1 | to taste
 Poulet Curry                            | pepper                         | pepper           | pinches |        1 | to taste
 Poulet Curry                            | cilantro leaves                | herb             | cups    |     0.33 | Fresh , for garnish (optional)
 Poulet Curry                            | basmati rice                   | rice             | cups    |        1 | Cooked , for serving
 Poulet Korma                            | chicken                        | chicken          | fillets |        4 | cut into pieces
 Poulet Korma                            | sunflower oil                  | oil              | tbsp    |        2 | 
 Poulet Korma                            | onions                         | onion            | pieces  |        2 | Medium , finely chopped
 Poulet Korma                            | garlic                         | garlic           | cloves  |        2 | minced
 Poulet Korma                            | ginger                         | ginger           | tbsp    |        1 |  Fresh, grated
 Poulet Korma                            | ground coriander               | spice            | tsp     |        2 | 
 Poulet Korma                            | ground cumin                   | spice            | tsp     |        1 | 
 Poulet Korma                            | ground turmeric                | spice            | tsp     |      0.5 | 
 Poulet Korma                            | red chili powder               | chili            | tsp     |      0.5 | (adjust to taste)
 Poulet Korma                            | tomato paste                   | tomato paste     | tbsp    |        1 | 
 Poulet Korma                            | yogurt                         | yogurt           | g       |      250 | plain, unsweetened
 Poulet Korma                            | heavy cream                    | heavy cream      | ml      |      150 | 
 Poulet Korma                            | salt                           | salt             | pinches |        1 | to taste
 Poulet Korma                            | fresh coriander leaves         | herb             | cups    |     0.33 |  for garnish
 Poulet Korma                            | basmati rice                   | rice             | cups    |        1 | for serving
 Poulet Tandoori                         | chicken thighs                 | chicken          | pieces  |        4 | 
 Poulet Tandoori                         | yogurt                         | yogurt           | g       |      200 | Plain
 Poulet Tandoori                         | garlic                         | garlic           | cloves  |        3 | 
 Poulet Tandoori                         | ginger                         | ginger           | pieces  |        1 | (about 2 cm piece)
 Poulet Tandoori                         | lemon juice                    | lemon            | tbsp    |        1 | 
 Poulet Tandoori                         | tandoori powder                | seasoning        | tsp     |        2 | 
Poulet Tandoori                         | cumin powder                   | spice            | tsp     |        1 | 
 Poulet Tandoori                         | coriander powder               | spice            | tsp     |        1 | 
 Poulet Tandoori                         | turmeric powder                | spice            | tsp     |        1 | 
 Poulet Tandoori                         | paprika                        | paprika          | tsp     |        1 | 
 Poulet Tandoori                         | red chili powder               | chili            | tsp     |      0.5 | (adjust to taste)
 Poulet Tandoori                         | salt                           | salt             | pinches |        1 |  to taste
 Poulet Tandoori                         | olive oil                      | olive oil        | tbsp    |        1 | 
 Quiche Lorraine                         | flour                          | flour            | g       |      200 | 
 Quiche Lorraine                         | butter                         | butter           | g       |      100 | softened and cut into small pieces
 Quiche Lorraine                         | salt                           | salt             | pinches |        1 | 
 Quiche Lorraine                         | water                          | water            | cl      |        5 | 
 Quiche Lorraine                         | bacon                          | bacon            | g       |      200 | Smoked  lardons
 Quiche Lorraine                         | eggs                           | egg              | pieces  |        4 | 
 Quiche Lorraine                         | thick cream                    | heavy cream      | ml      |      200 | 
 Quiche Lorraine                         | milk                           | milk             | ml      |      200 | 
 Quiche Lorraine                         | pepper                         | pepper           | pinches |        1 | 
 Quiche Lorraine                         | nutmeg                         | nutmeg           | pinches |        1 | 
 Ratatouille                             | aubergine                      | aubergine        | pieces  |        1 | 
 Ratatouille                             | courgette                      | zucchini         | pieces  |        1 | 
 Ratatouille                             | bell pepper                    | bell pepper      | pieces  |        2 | Red  and Yellow
 Ratatouille                             | onion                          | onion            | pieces  |        1 | 
 Ratatouille                             | garlic                         | garlic           | cloves  |        2 | 
 Ratatouille                             | tomatoes                       | tomato           | pieces  |        4 | Ripe
 Ratatouille                             | thyme                          | herb             | sprig   |        1 | 
 Ratatouille                             | bay                            | bay              | leaves  |        1 | 
 Ratatouille                             | olive oil                      | olive oil        | tbsp    |        3 | 
 Ratatouille                             | salt                           | salt             | pinches |        1 | 
 Ratatouille                             | black pepper                   | pepper           | pinches |        1 |  freshly ground
 Ratatouille                             | basil                          | basil            | leaves  |       15 | for garnish (optional)
 Salade César au poulet grillé           | bread                          | bread            | pieces  |        2 | Large pita bread or white bread
 Salade César au poulet grillé           | olive oil                      | olive oil        | tbsp    |        2 | 
 Salade César au poulet grillé           | romaine lettuce                | lettuce          | heads   |        2 | Washed and torn into pieces
 Salade César au poulet grillé           | parmesan                       | cheese           | g       |       60 | Grated
 Salade César au poulet grillé           | chicken                        | chicken          | fillets |        2 | Skinless
 Salade César au poulet grillé           | garlic                         | garlic           | cloves  |        1 | Chopped
 Salade César au poulet grillé           | anchovy                        | anchovy          | fillets |        3 | 
 Salade César au poulet grillé           | egg yolks                      | egg yolk         | pieces  |        2 | 
 Salade César au poulet grillé           | dijon mustard                  | mustard          | tbsp    |        1 | 
 Salade César au poulet grillé           | lemon juice                    | lemon            | tbsp    |        2 | Fresh
 Salade César au poulet grillé           | salt                           | salt             | pinches |        1 | 
 Salade César au poulet grillé           | pepper                         | pepper           | pinches |        1 | 
 Salade César au poulet grillé           | olive oil                      | olive oil        | ml      |      120 | 
 Salade César classique                  | bread                          | bread            | pieces  |        2 | Large pita bread or white bread
 Salade César classique                  | olive oil                      | olive oil        | tbsp    |        2 | 
 Salade César classique                  | romaine lettuce                | lettuce          | heads   |        2 | Washed and torn into pieces
 Salade César classique                  | parmesan                       | cheese           | g       |       60 | Grated
 Salade César classique                  | garlic                         | garlic           | cloves  |        1 | Chopped
 Salade César classique                  | anchovy                        | anchovy          | fillets |        3 | 
 Salade César classique                  | egg yolks                      | egg yolk         | pieces  |        2 | 
 Salade César classique                  | mustard                        | mustard          | tbsp    |        1 | Dijon
 Salade César classique                  | lemon juice                    | lemon            | tbsp    |        2 | Fresh
 Salade César classique                  | salt                           | salt             | pinches |        1 | 
 Salade César classique                  | pepper                         | pepper           | pinches |        1 | 
 Salade César classique                  | olive oil                      | olive oil        | ml      |      120 | 
 Spaghetti Carbonara                     | spaghetti                      | pasta            | g       |      200 | 
 Spaghetti Carbonara                     | guanciale                      | bacon            | g       |      100 | (or lardons or diced pancetta)
 Spaghetti Carbonara                     | egg yolks                      | egg yolk         | pieces  |        2 | Large
 Spaghetti Carbonara                     | pecorino                       | cheese           | g       |       50 | Grated
 Spaghetti Carbonara                     | parmesan                       | cheese           | g       |       50 | Grated
 Spaghetti Carbonara                     | black pepper                   | pepper           | pinches |        1 | Freshly ground
 Spaghetti Carbonara                     | salt                           | salt             | pinches |        1 | 
 Spanish Cod with Tomatoes and Olives    | salt                           | salt             | pinches |        1 | 
Spanish Cod with Tomatoes and Olives    | pepper                         | pepper           | pinches |        1 | freshly ground
 Spanish Cod with Tomatoes and Olives    | potatoes                       | potato           | g       |      350 | Small red potatoes, quartered
 Spanish Cod with Tomatoes and Olives    | cod                            | cod              | g       |      800 | 
 Spanish Cod with Tomatoes and Olives    | olive oil                      | olive oil        | tbsp    |        1 | Extra virgin
 Spanish Cod with Tomatoes and Olives    | paprika                        | paprika          | tsp     |        1 | Smoked or regular
 Spanish Cod with Tomatoes and Olives    | onion                          | onion            | pieces  |        1 | medium shallot or ½ small red onion, thinly sliced
 Spanish Cod with Tomatoes and Olives    | red peppers                    | bell pepper      | cups    |      0.5 | Thickly-sliced, jarred roasted
 Spanish Cod with Tomatoes and Olives    | garlic                         | garlic           | cloves  |        1 | very thinly sliced
 Spanish Cod with Tomatoes and Olives    | dry white wine                 | wine             | cups    |     0.25 | 
 Spanish Cod with Tomatoes and Olives    | tomatoes                       | tomato           | g       |      450 | Stewed  with their juices (or whole plum tomatoes, coarsely chopped)
 Spanish Cod with Tomatoes and Olives    | green olives                   | olives           | cups    |      0.5 | Pitted
 Spanish Cod with Tomatoes and Olives    | bread                          | bread            | g       |      100 | Crusty , for serving
 Sushi Maki                              | sushi rice                     | rice             | cups    |        2 | (short-grain round rice)
 Sushi Maki                              | water                          | water            | cups    |      2.5 | 
 Sushi Maki                              | rice vinegar                   | vinegar          | cups    |      0.5 | 
 Sushi Maki                              | sugar                          | sugar            | tbsp    |        3 | 
 Sushi Maki                              | salt                           | salt             | tsp     |        2 | 
 Sushi Maki                              | nori sheets                    | nori sheets      | sheets  |       10 | (dried seaweed)
 Sushi Maki                              | cucumber                       | cucumber         | pieces  |        1 | 
 Sushi Maki                              | avocado                        | avocado          | pieces  |        1 | 
 Sushi Maki                              | fresh sushi fish               | fish             | g       |      200 |  Fresh tuna or salmon
 Sushi Maki                              | soy sauce                      | soy sauce        | ml      |       50 | for dipping
 Sushi Maki                              | wasabi                         | wasabi           | g       |       15 | 
 Sushi Maki                              | ginger                         | ginger           | g       |       15 | pickled, to serve
 Tarte aux pommes classique              | flour                          | flour            | g       |      200 | 
 Tarte aux pommes classique              | butter                         | butter           | g       |      100 | softened and cut into small pieces
 Tarte aux pommes classique              | salt                           | salt             | pinches |        1 | 
 Tarte aux pommes classique              | water                          | water            | cl      |        5 | 
 Tarte aux pommes classique              | apples                         | apple            | pieces  |        7 | 
 Tarte aux pommes classique              | sugar                          | sugar            | g       |      100 | 
 Tarte aux pommes classique              | cinnamon                       | cinnamon         | tsp     |        2 | optional
 Tarte aux pommes classique              | lemon juice                    | lemon            | tbsp    |        1 | 
 Tarte aux pommes classique              | flour                          | flour            | tbsp    |        2 | 
 Tarte Tatin                             | flour                          | flour            | g       |      200 | 
 Tarte Tatin                             | butter                         | butter           | g       |      100 | softened and cut into small pieces
 Tarte Tatin                             | salt                           | salt             | pinches |        1 | 
 Tarte Tatin                             | water                          | water            | cl      |        5 | 
 Tarte Tatin                             | apples                         | apple            | pieces  |        7 | 
 Tarte Tatin                             | sugar                          | sugar            | g       |      150 | 
 Tarte Tatin                             | cinnamon                       | cinnamon         | tsp     |        1 | optional
 Tomato Risotto                          | onion                          | onion            | pieces  |        1 | Small
 Tomato Risotto                          | arborio rice                   | rice             | cups    |      1.5 | 
 Tomato Risotto                          | dry white wine                 | wine             | cups    |      0.5 | 
 Tomato Risotto                          | tomato sauce                   | tomato sauce     | cups    |        1 | 
 Tomato Risotto                          | butter                         | butter           | tbsp    |        1 | Unsalted
 Tomato Risotto                          | parmesan                       | cheese           | tbsp    |        2 | Grated
 Tomato Risotto                          | oil-poached tomatos            | dried tomatos    | pieces  |        6 |  Coarsely chopped
 Tomato Risotto                          | olive oil                      | olive oil        | tbsp    |        2 | 
 Tomato Risotto                          | salt                           | salt             | pinches |        1 | Kosher, to taste
 Tomato Risotto                          | black pepper                   | pepper           | pinches |        1 | Freshly ground, to taste
 Tomato Risotto with Saffron             | tomatoes                       | tomato           | g       |      500 | Cherry or grape
 Tomato Risotto with Saffron             | onion                          | onion            | pieces  |        1 | White or yellow
 Tomato Risotto with Saffron             | olive oil                      | olive oil        | tbsp    |        2 | 
 Tomato Risotto with Saffron             | garlic                         | garlic           | cloves  |        5 | 
 Tomato Risotto with Saffron             | thyme                          | herb             | tsp     |        1 | Dried (or fresh)
 Tomato Risotto with Saffron             | arborio rice                   | rice             | cups    |      1.5 | or Spanish short-grain rice
 Tomato Risotto with Saffron             | saffron                        | spice            | pinches |        1 | 
 Tomato Risotto with Saffron             | salt                           | salt             | tsp     |      0.5 | 
 Tomato Risotto with Saffron             | pepper                         | pepper           | tsp     |      0.5 | 
 Tomato Risotto with Saffron             | paprika                        | paprika          | tsp     |     0.25 | Smoked
 Tomato Risotto with Saffron             | broth                          | broth            | cups    |        7 | Veggie stock or chicken stock
 Tomato Risotto with Saffron             | butter                         | butter           | tbsp    |        1 | 
 Tomato Risotto with Saffron             | parmesan                       | cheese           | cups    |     0.25 | Grated
(371 rows)
```